# Proyecto EL7008 | Primavera 2021
## Aprendizaje semi-supervisado basado en FixMatch
#### Paper original por Kihyuk Sohn y David Berthelot en [arXiv](https://arxiv.org/abs/2001.07685)

---
**Autor:** ***Tomás Rodrigo Saldivia Astudillo***  

---

En este proyecto se busca elaborar un clasificador de imágenes siguiendo el paradigma de aprendizaje semi-supervisado (SSL), en particular utilizando el algoritmo FixMatch. El aprendizaje semi-supervisado consiste en combinar una pequeña cantidad de datos etiquetados con una gran cantidad de datos no-etiquetados para entrenar un modelo, y FixMatch en particular lo hace generando una pseudo-label para datos no-etiquetados mediante consistency regularization.  

En el proyecto se implementa este algoritmo y se aplica en CIFAR-10 y
CIFAR-100. Se prueban dos arquitecturas distintas, ambas variantes de WideResNet y se utilizan dos esquemas de data augmentation fuertes distintos para comparar con sus resultados, siendo uno de estos RandAugment.  

Se analiza y cuantifica el efecto que tienen los datos no-etiquetados en el entrenamiento en cada caso, y además se compara con el caso sin FixMatch. El proyecto esta desarrollado en python utilizando Pytorch.

***RandAugment***: [paper en arXiv](https://arxiv.org/abs/1909.13719)

## Código

El código implementado se encuentra en el repositorio [github:Tom0497/FixMatchEL7008](https://github.com/Tom0497/FixMatchEL7008) y cuenta con scripts para ejecutar los distintos tipos de entrenamientos asociados a los experimentos que se desean realizar.

In [ ]:
!git clone https://github.com/Tom0497/FixMatchEL7008.git
%cd FixMatchEL7008/

Es imperativo agregar el directorio raíz del proyecto a la variable de entorno de python, de esta manera, las exportaciones de los distintos modulos funcionaran de la manera correcta. Se puede tomar la siguiente celda, que imprime el current working directory, y luego modificar la variable de entorno PYTHONPATH para que la contenga. La celda de más abajo funciona en PaperSpace, pero en colab se debiera cambiar `os.environ['PYTHONPATH'] =` por `os.environ['PYTHONPATH'] +=`.

In [ ]:
import os

print(os.getcwd())

In [ ]:
!echo $PYTHONPATH

In [ ]:
import os

os.environ['PYTHONPATH'] = "/notebooks/FixMatchEL7008:"
!echo $PYTHONPATH

### Requerimientos

Para la correcta ejecución del código se requiere de pytorch en su última versión, en específico torchvision 0.11.2 o más. Además, se utiliza sklearn, matplotlib, numpy.

In [ ]:
!pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

### Entrenamiento supervisado sobre CIFAR

El script `supervised.py` permite la ejecución de un entrenamiento totalmente supervisado sobre CIFAR10 o CIFAR100. Gran parte de los hiper parámetros de entrenamiento son ajustables, como también lo es la arquitectura del modelo WideResNet utilizar. Se muestran a continuación las opciones disponibles para ejecutar el script.

```{bash}
usage: supervised.py [-h] [-d {cifar10,cifar100}] [-e EPOCHS] [-bs BATCH_SIZE]
                     [-md MODEL_DEPTH] [-mw MODEL_WIDTH] [-es EARLY_STOPPING]
                     [-r RESULTS] [-tr TRAIN_RANGE TRAIN_RANGE]
                     [-vr VAL_RANGE VAL_RANGE]

optional arguments:
  -h, --help            show this help message and exit
  -d {cifar10,cifar100}, --data {cifar10,cifar100}
                        dataset for training
  -e EPOCHS, --epochs EPOCHS
                        number of epochs for training
  -bs BATCH_SIZE, --batch-size BATCH_SIZE
                        batch size for training
  -md MODEL_DEPTH, --model-depth MODEL_DEPTH
                        depth of Wide ResNet model
  -mw MODEL_WIDTH, --model-width MODEL_WIDTH
                        width of Wide ResNet model
  -es EARLY_STOPPING, --early-stopping EARLY_STOPPING
                        number of epochs for early stopping
  -r RESULTS, --results RESULTS
                        folder name for training results
  -tr TRAIN_RANGE TRAIN_RANGE, --train-range TRAIN_RANGE TRAIN_RANGE
                        range of images per class for training
  -vr VAL_RANGE VAL_RANGE, --val-range VAL_RANGE VAL_RANGE
                        range of images per class for validation
```


### Entrenamiento semi-supervisado con FixMatch sobre CIFAR  

El script `fixmatch.py` permite la ejecución de un entrenamiento semi-supervisado sobre CIFAR10 o CIFAR100. Gran parte de los hiper parámetros de entrenamiento son ajustables, como también lo es la arquitectura del modelo WideResNet utilizar. Se muestran a continuación las opciones disponibles para ejecutar el script.

```{bash}
usage: fixmatch.py [-h] [-d {cifar10,cifar100}] [-e EPOCHS] [-bs BATCH_SIZE]
                   [-md MODEL_DEPTH] [-mw MODEL_WIDTH] [-es EARLY_STOPPING]
                   [-r RESULTS] [-tr TRAIN_RANGE TRAIN_RANGE]
                   [-vr VAL_RANGE VAL_RANGE]
                   [-ulr UNLABELED_RANGE UNLABELED_RANGE] [-tau TAU] [-mu MU]
                   [--lambda-u LAMBDA_U] [-N N] [-M M]

optional arguments:
  -h, --help            show this help message and exit
  -d {cifar10,cifar100}, --data {cifar10,cifar100}
                        dataset for training
  -e EPOCHS, --epochs EPOCHS
                        number of epochs for training
  -bs BATCH_SIZE, --batch-size BATCH_SIZE
                        batch size for training
  -md MODEL_DEPTH, --model-depth MODEL_DEPTH
                        depth of Wide ResNet model
  -mw MODEL_WIDTH, --model-width MODEL_WIDTH
                        width of Wide ResNet model
  -es EARLY_STOPPING, --early-stopping EARLY_STOPPING
                        number of epochs for early stopping
  -r RESULTS, --results RESULTS
                        folder name for training results
  -tr TRAIN_RANGE TRAIN_RANGE, --train-range TRAIN_RANGE TRAIN_RANGE
                        range of images per class for training
  -vr VAL_RANGE VAL_RANGE, --val-range VAL_RANGE VAL_RANGE
                        range of images per class for validation
  -ulr UNLABELED_RANGE UNLABELED_RANGE, --unlabeled-range UNLABELED_RANGE UNLABELED_RANGE
                        range of images per class for unlabeled data
  -tau TAU, --tau TAU   threshold for retaining a pseudo-label
  -mu MU, --mu MU       multiplier of batch size for unlabeled data
  --lambda-u LAMBDA_U   unsupervised loss multiplier lambda
  -N N, --N N           number of transformations for RandAugment
  -M M, --M M           magnitude of transformations in RandAugment
```

## Baseline sin Fixmatch utilizando todo el dataset

Este experimento busca tener un baseline de comparación, para ello se entrena en primer lugar sobre todo el dataset de CIFAR10 en un esquema supervisado. De las 5000 imágenes por clases de CIFAR10, 1000 se ocupan para evaluar la red y las restantes 4000 se utilizan para entrenar. Los hiperparámetros a utilizar son:



In [ ]:
!python src/supervised.py -d cifar10 -e 100 -bs 512 -md 22 -mw 2 -es 15 -r baseline1 -tr 0 4000 -vr 4000 5000

## Baseline sin FixMatch utilizando porción restringida de entrenamiento
En este experimento se entrena sobre parte reducida del dataset de entrenamiento, de forma de evidenciar las limitaciones de entrenar con pocos datos.

In [ ]:
!python src/supervised.py -d cifar10 -e 300 -bs 256 -md 22 -mw 2 -es 15 -r baseline2 -tr 0 400 -vr 4000 5000

## Fixmatch con RandAugment(N=2, M=9)

En este experimento se entrena con fixmatch utilizando parte reducida del dataset de forma etiquetada y la totalidad del dataset de forma no etiquetada. El esquema de data augmentation fuerte es RandAugment con parámetros N=2 y M=9.

In [ ]:
!python src/fixmatch.py -d cifar10 -e 100 -bs 128 -md 22 -mw 2 -es 15 -r fixmatch1 -tr 0 400 -vr 4000 5000 -ulr 0 4000

## Fixmatch con RandAugment(N=3, M=4)

En este experimento se entrena con fixmatch utilizando parte reducida del dataset de forma etiquetada y la totalidad del dataset de forma no etiquetada. El esquema de data augmentation fuerte es RandAugment con parámetros N=3 y M=4.

In [ ]:
!python src/fixmatch.py -d cifar10 -e 100 -bs 128 -md 22 -mw 2 -es 15 -r fixmatch2 -tr 0 400 -vr 4000 5000 -ulr 0 4000 -N 3 -M 4

## Baseline sin FixMatch utilizando porción restringida de entrenamiento
En este experimento se entrena sobre parte reducida del dataset de entrenamiento, de forma de evidenciar las limitaciones de entrenar con pocos datos. En este caso se ha cambiado el modelo de WRN-22-2 a WRN-16-2.

In [ ]:
!python src/supervised.py -d cifar10 -e 300 -bs 256 -md 16 -mw 2 -es 15 -r baseline3 -tr 0 400 -vr 4000 5000

## Fixmatch con RandAugment(N=2, M=9) WRN-16-2

En este experimento se entrena con fixmatch utilizando parte reducida del dataset de forma etiquetada y la totalidad del dataset de forma no etiquetada. El esquema de data augmentation fuerte es RandAugment con parámetros N=2 y M=9. El modelo se ha cambiado de WRN-22-2 a WRN-16-2.

In [ ]:
!python src/fixmatch.py -d cifar10 -e 100 -bs 128 -md 16 -mw 2 -es 15 -r fixmatch3 -tr 0 400 -vr 4000 5000 -ulr 0 4000

## Baseline sin FixMatch utilizando porción restringida de entrenamiento
En este experimento se entrena sobre parte reducida del dataset de entrenamiento, de forma de evidenciar las limitaciones de entrenar con pocos datos. En este caso, se utiliza CIFAR100

In [ ]:
!python src/supervised.py -d cifar100 -e 300 -bs 512 -md 22 -mw 2 -es 15 -r baseline4 -tr 0 100 -vr 400 500

## Fixmatch con RandAugment(N=2, M=9) WRN-22-2 CIFAR100

En este experimento se entrena con fixmatch utilizando parte reducida del dataset de forma etiquetada y la totalidad del dataset de forma no etiquetada. El esquema de data augmentation fuerte es RandAugment con parámetros N=2 y M=9. Se ha cambiado de CIFAR 10 a CIFAR 100.

In [ ]:
!python src/fixmatch.py -d cifar100 -e 300 -bs 128 -md 22 -mw 2 -es 15 -r fixmatch4 -tr 0 100 -vr 400 500 -ulr 0 400

## Resultados
Todos los resultados, logs, modelos, etc. quedan almacenados en `FixMatchEL7008/runs`. Luego, se pueden descargar para ser procesados localmente, o en este mismo notebook de manera directa. Además se provee funcionalidad para generar los gráficos relevantes de cada entrenamiento y testeo. La siguiente celda ejemplifica el uso para uno de los entrenamientos, pero modificando la variable `folder` se puede utilizar para cualquiera de los resultados obtenidos.

In [ ]:
from definitions import RUNS_DIR
from src.train.summary.plotter import Plotter

folder = 'fixmatch1'
path = RUNS_DIR / folder
num_classes = 100 if folder in ['baseline4', 'fixmatch4'] else 10
plttr = Plotter(base_path=path, n_classes=num_classes)

plttr.print_metadata(fixmatch='fixmatch' in folder)

plttr.plot_loss(save=path / 'train_loss.png')
plttr.plot_accu(save=path / 'train_accu.png')
plttr.test_best(save=path / 'conf_matrix.png')

if 'fixmatch' in folder:
  plttr.plot_ssl_metrics(save=path / 'ssl_metrics.png')
  plttr.plot_ssl_losses(save=path / 'ssl_losses.png')